<h1 align="center">Mushrooms</h1>
<h5 align="center">by</h5>
<h3 align="center">Monde Anna</h3>

<br />
<br />


<h3 align="center">Edible or Not</h3>
<h5 align="center">Predicted with the Aid of Decision Trees</h5>

<br />

<p>We will train a <b><i>Decision Tree</i></b> model on 23 species of gilled mushrooms in the Agaricus and Lepiota Family and further test the accuracy of said model.</p>

<br />
<br />


In [1]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np


<br />

<h3 align="center">Globals</h3>

<br />
<br />


In [2]:
SEED = 42
SERIES = pd.core.series.Series
